En repartant du dataset "people.csv" (cf. lesson4) dans sa version "clean" finale:

- mettre la colonne `inactive` à `true` pour tous les users dont le `last_seen` date d'au moins un an
- avec une regex: filtrer les numéros de téléphone invalides
- ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)
- ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"

In [2]:
import pandas as pd

# if you don't want pandas.read_csv to mess with data types,
# you can force it to keep str values by specifying dtype=str.
people = pd.read_csv('people.csv')

In [30]:
def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

df_clean = clean_people(people)

In [31]:
df_clean.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic


In [37]:
import datetime

test = df.copy()

is_inactive = df_clean.last_seen < datetime.datetime.now() \
    .replace(year=stop_date.year-1)

is_inactive = df_clean.last_seen < \
    (datetime.datetime.now() - datetime.timedelta(days=365))
test['inactive'] = is_inactive

test.head(20)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
6,27539,Dulcine,Stopforth,dstopforth31@forbes.com,F,22.0,13.860,-35.587670,-6.255393,NaN,2017-12-13,False,2019-01-15 15:39:05,"Serrinha, Brazil",entrée/plat/dessert,Dulcine Stopforth,Brazil
7,27469,Denny,Hasnip,dhasnip13@so-net.ne.jp,F,47.0,83.567,-89.917591,14.113818,NaN,2017-05-01,False,2019-03-08 18:46:46,"Comapa, Guatemala",entrée/plat/dessert,Denny Hasnip,Guatemala
9,27565,Esmeralda,Hollindale,ehollindale3r@guardian.co.uk,F,29.0,60.590,-36.508738,-54.281149,NaN,2017-10-05,True,2018-04-12 15:45:51,"Grytviken, China",entrée/plat/dessert,Esmeralda Hollindale,China
10,27446,Skippie,Arlott,sarlottg@so-net.ne.jp,M,49.0,27.797,38.954230,55.877840,NaN,2017-08-13,True,2018-10-06 10:55:59,"Malaya Dubna, Russia",entrée/plat/dessert,Skippie Arlott,Russia
11,27550,Shermie,Locard,slocard3c@lycos.com,M,65.0,35.948,112.813530,-7.305958,NaN,2018-09-29,False,2019-05-03 00:23:45,"Wonorejo, Indonesia",entrée/plat/dessert,Shermie Locard,Indonesia


In [38]:
import re
pattern = '?:\B\+ ?49|\b0)(?: *[(-]? *\d(?:[ \d]*\d)?)? *(?:[)-] *)?\d+ *(?:[/)-] *)?\d+ *(?:[/)-] *)?\d+(?: *- *\d+)?'

pattern = '^0\d{9}$'
df_clean.phone.str.contains(pattern, regex=True)

0      True
1       NaN
2       NaN
3       NaN
5      True
       ... 
204     NaN
205     NaN
206    True
207    True
208    True
Name: phone, Length: 191, dtype: object